In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

DEBUG:__main__:hello


In [2]:
benchmark_name = "cg"

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

dict_column_names_JP = {
    "functionName": "関数名",
    "modelLin": "線形",
    "modelIp": "反比例",
    "modelLog": "対数",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    "modelIpAndLin": "反比例＆線形",
    "modelIpAndLog": "反比例＆対数",
    "modelLogAndLin": "対数＆線形",
    "modelLogAndIp": "対数＆反比例",
}

model_names_list_JP = list(dict_column_names_JP.values())
model_names_list_JP.remove("関数名")

model_names_list = list(dict_column_names_JP.keys())
model_names_list.remove("functionName")

date = "2022年2月2日"

# コア数と問題サイズの両方を振る

rawDF = return_rawDF_with_init_param(
    benchmark_name=benchmark_name,
    classes=classes,
    processes=processes,
    csv_dir_path="./csv_files/",
)

exp_var = rawDF.columns.tolist()
for element_be_removed in [
    "functionName",
    "functionCallNum",
    "intBenchmarkClass",
    "benchmarkName",
    "benchmarkClass",
]:
    exp_var.remove(element_be_removed)
res_var = ["functionCallNum"]

rawDF_CONJ_GRAD = rawDF[rawDF["functionName"] == "CONJ_GRAD"]
rawDF_CONJ_GRAD

# returned_MAPE_table = return_MAPE_Table_DF_from_rawDF(
#     rawDF=rawDF,
#     exp_var_list=exp_var,
#     res_var_list=res_var,
#     model_name_list=model_names_list,
# )
# df = returned_MAPE_table.rename(columns=dict_column_names_JP)
# df = df.set_index("関数名")
# df = df.sort_index(axis=0)
# df = df.sort_index(axis=1)

# df = addLowestMAPEsModelNameColumn(df, version=2, model_name_list=model_names_list_JP)

# print(
#     df.to_latex(
#         caption=f"ベンチマークプログラム{benchmark_name.upper()}における各モデルでのMAPE",
#         label=f"{date}everyModelsMAPEtableIn{benchmark_name.upper()}",
#     )
# )

,functionName,functionCallNum,benchmarkName,benchmarkClass,process,intBenchmarkClass,na,nonzer,niter,shift
5,CONJ_GRAD,16.0,cg,A,2,1,14000,11,15,20
5,CONJ_GRAD,16.0,cg,A,4,1,14000,11,15,20
5,CONJ_GRAD,16.0,cg,A,8,1,14000,11,15,20
5,CONJ_GRAD,16.0,cg,A,16,1,14000,11,15,20
5,CONJ_GRAD,16.0,cg,A,32,1,14000,11,15,20
5,CONJ_GRAD,16.0,cg,A,64,1,14000,11,15,20
5,CONJ_GRAD,16.0,cg,A,128,1,14000,11,15,20
6,CONJ_GRAD,16.0,cg,A,256,1,14000,11,15,20
2,CONJ_GRAD,31.0,cg,B,2,4,75000,13,75,60
2,CONJ_GRAD,46.0,cg,B,4,4,75000,13,75,60


# (2022年2月23日~) 複数説明変数によるcurve_fitの実験

In [3]:
# TODO: 👆やる

# 説明変数
plotX = np.linspace(10, 20, 10)
plotY = np.linspace(10, 20, 10)
plotZ = np.linspace(10, 20, 10)
# 目的変数
a = random.uniform(1, 30)
b = random.uniform(1, 30)
c = random.uniform(1, 30)
d = random.uniform(1, 30)
plotT = (a * plotX) / (b * plotY + c * plotZ) + d

# DFを作成する
# カラム名のリスト
columnNames = ["plotX", "plotY", "plotZ", "plotT"]
datumForDF = [plotX, plotY, plotZ, plotT]
inputDFForTest = pd.DataFrame(index=columnNames, data=datumForDF).T
inputDFForTest["functionName"] = "functionName"

# 目的変数・説明変数のカラム名のリスト
# 目的変数のカラム名のリスト
columnNamesForExp = ["plotX", "plotY", "plotZ"]
# 説明変数のカラム名のリスト
columnNamesForRes = ["plotT"]

# （2022年2月23日～）

# 新モデルの実装

In [4]:
# TODO:新モデルに変更する


class Model_ProcessesDevidedByProblemSize_ForMultipleRegression(
    ModelBaseForMultipleRegression
):
    """プロセス数を問題サイズの変数群の線形和で割ったモデル

    組み合わせモデルを実現するためのクラス

    Attributes:
        equationDict (dict): キー・バリューが列名・変形モデル（線形、反比例、対数など）
        lr : モデルのオブジェクト
        dataXForPredict : 説明変数のDF
        dataTForPredict : 目的変数のDF
    Note:
        モデルの式は次の通り
        関数コール回数 = (プロセス数)/(a * 問題サイズ1 + b * 問題サイズ2 + ... + n * 問題サイズn + c)
    """
    
    def build_model(self):
        # TODO:モデル式と同様の関数を宣言
        # TODO:curve_fit()
        # 

    def predict(self, inputDF):
        """predict(self, inputDF)

        inputDFのデータから構築されたモデルを使って予測を行う

        Args:
            self : none
            inputDF (pandas.DataFrame): 構築されたモデルを用いて予測に使うDF

        Returns:
            pandas.DataFrame: 構築されたモデルから予測された値。型に確証なし
        """
        
        # TODO:predict()を必要に応じて実装する

    def returnMAPE(self):
        """calcMAPE(self)

        モデルの構築に使用されたデータからMAPEを算出する

        Args:
            self : none

        Returns:
            list: モデルの構築に用いたデータから予測された値
            int: 失敗した場合、-1
        """
        
        # TODO:returnMAPE()を必要に応じて実装する
        


def test_Model_ProcessesDevidedByProblemSize_ForMultipleRegression():
    """test_ModelMultipleEquationForMultipleRegression()

    ModelMultipleEquationForMultipleRegressionのテスト
    """

    # 説明変数
    plotX = np.linspace(10, 20, 10)
    plotY = np.linspace(10, 20, 10)
    plotZ = np.linspace(10, 20, 10)
    # 目的変数
    a = random.uniform(1, 30)
    b = random.uniform(1, 30)
    c = random.uniform(1, 30)
    d = random.uniform(1, 30)
    plotT = (a * plotX) / (b * plotY + c * plotZ) + d

    # DFを作成する
    # カラム名のリスト
    columnNames = ["plotX", "plotY", "plotZ", "plotT"]
    datumForDF = [plotX, plotY, plotZ, plotT]
    inputDFForTest = pd.DataFrame(index=columnNames, data=datumForDF).T
    inputDFForTest["functionName"] = "functionName"

    # 目的変数・説明変数のカラム名のリスト
    # 目的変数のカラム名のリスト
    columnNamesForExp = ["plotX", "plotY", "plotZ"]
    # 説明変数のカラム名のリスト
    columnNamesForRes = ["plotT"]

    # 予測をする
    # モデルオブジェクトの作成
    objectModel = Model_ProcessesDevidedByProblemSize_ForMultipleRegression(
        inputDF=inputDFForTest,
        explanatoryVariableColumnNames=columnNamesForExp,
        responseVariableColumnNames=columnNamesForRes,
        conditionDictForTest={},
    )
    # TODO:モデルの構築

    # モデル構築に用いたデータとのMAPEによって実装がうまくいっているかどうかの判定を行う
    MAPE = objectModel.returnMAPE()
    print(f"MAPE = {MAPE}")
    assert MAPE < 1

IndentationError: expected an indented block after function definition on line 21 (4055975222.py, line 26)

In [ ]:
test_Model_ProcessesDevidedByProblemSize_ForMultipleRegression()

# （2022年2月22日～）【個人作業】

# 回帰木による予測

In [ ]:
from sklearn.tree import DecisionTreeRegressor

rawDF

In [ ]:
rawDF_CONJ_GRAD = rawDF[rawDF["functionName"] == "CONJ_GRAD"]
rawDF_CONJ_GRAD

In [ ]:
print(f"exp_var = {exp_var}")

print(f"res_var = {res_var}")

print(f"rawDF_CONJ_GRAD.shape = {rawDF_CONJ_GRAD.shape}")

In [ ]:
X = rawDF_CONJ_GRAD.filter(items=exp_var)
Y = rawDF_CONJ_GRAD.filter(items=res_var)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.1, random_state=1234
)

# X_train, X_test, Y_train, Y_test

In [ ]:
reg = DecisionTreeRegressor(max_leaf_nodes=20)

model = reg.fit(X_train, Y_train)
print(f"model = {model}")

In [ ]:
from sklearn.metrics import r2_score

YHat = model.predict(X_test)
r2 = r2_score(Y_test, YHat)
print("R^2 = ", r2)

In [ ]:
from sklearn import tree
from IPython.display import Image
import pydotplus

dot_data = tree.export_graphviz(
    model,
    out_file=None,
    feature_names=rawDF_CONJ_GRAD.feature_names,
    class_names="functionCallNum",
    filled=True,
)
graph = pydotplus.graph_from_dot_data(dot_data)

Image(graph.create_png())